In [1]:
from Strategies.StrategyBB import StrategyBB
from Strategies.StrategyMACD import StrategyMACD
from Strategies.StrategyOBV import StrategyOBV
from Strategies.StrategyRSI import StrategyRSI
from Strategies.StrategyST import StrategyST
from historyData.HistoryData import HistoryData
from tinkoff.invest.grpc.marketdata_pb2 import CandleInterval
from datetime import datetime

import asyncio

In [2]:
candle_interval = CandleInterval.CANDLE_INTERVAL_1_MIN
data_set = HistoryData()

In [3]:
from time import sleep

candles_down = None
candles_up = None
candles_flat = None
candles_short_up = None
candles_short_down = None

async def data_get():
    global candles_down, candles_up, candles_flat, candles_short_up, candles_short_down
    
    candles_short_up = await (data_set.get_tinkoff_server_data(datetime(2023, 1, 1), datetime(2023, 2, 1), candle_interval))
    candles_short_down = await (data_set.get_tinkoff_server_data(datetime(2021, 1, 1), datetime(2021, 2, 1), candle_interval))
    candles_down = await (data_set.get_tinkoff_server_data(datetime(2022, 2, 1), datetime(2022, 3, 1), candle_interval))
    candles_up = await (data_set.get_tinkoff_server_data(datetime(2023, 3, 1), datetime(2023, 4, 1), candle_interval))
    candles_flat = await (data_set.get_tinkoff_server_data(datetime(2024, 1, 1), datetime(2024, 2, 1), candle_interval))

# Run the event loop directly using await
await data_get()

# Creates a Data Set For Future Model

In [4]:
all_candles = [candles_down, candles_up, candles_flat, candles_short_up, candles_short_down]
len(all_candles[3])

21830

In [5]:
from AppLogic.Settings import Settings
from tinkoff.invest.utils import quotation_to_decimal

commission = 0.0005
all_data_index_buy_sell = []

for candles in all_candles:
    data_index_buy_sell = []
    total_earn = 0
    total_sum_earned = 0
    bought_at = -1
    for i, candle in enumerate(list(candles)):
        bought_at = float(quotation_to_decimal(candle.close))
        this_candle_earned = 0
        for j in range(i + 1, len(candles)):
            further_candle_price = float(quotation_to_decimal(candles[j].close))
            this_candle_earned = bought_at - further_candle_price
            if this_candle_earned > 0.1 / 100 * bought_at:  # 0.1 is stop loss
                data_index_buy_sell.append({"bought": i})
                break
            elif this_candle_earned < -0.05 / 100 * bought_at:  # 0.05 is stop loss
                break
            
    all_data_index_buy_sell.append(data_index_buy_sell)
    print(f"Length of the marked data: {len(data_index_buy_sell)}", f"total length: {len(candles)}")

Length of the marked data: 7335 total length: 17378
Length of the marked data: 8953 total length: 22663
Length of the marked data: 8672 total length: 21769
Length of the marked data: 8530 total length: 21830
Length of the marked data: 5902 total length: 15422


In [6]:
import pandas as pd

all_new_data_buy = list()
for data_index_buy_sell in all_data_index_buy_sell:
    new_data_buy = list()
    for i, data in enumerate(data_index_buy_sell):
        if list(data_index_buy_sell[i].keys())[0] != "sold":
            new_data_buy.append(data)
    all_new_data_buy.append(new_data_buy)

all_df_data = list()

for c_i, candles in enumerate(all_candles):
    df_data = {"index": [], "result": [], "close": [],
               "open": [], "high": [], "low": [],
               "volume": []}
    buy_counter = 0
    for i, candle in enumerate(list(candles)):
        df_data["index"].append(i)
        df_data["close"].append(float(quotation_to_decimal(candle.close)))
        df_data["open"].append(float(quotation_to_decimal(candle.open)))
        df_data["high"].append(float(quotation_to_decimal(candle.high)))
        df_data["low"].append(float(quotation_to_decimal(candle.low)))
        df_data["volume"].append(candle.volume)
        
        if i == all_new_data_buy[c_i][buy_counter]["bought"]:
            df_data["result"].append(1)
            buy_counter += 1 if buy_counter < len(all_new_data_buy[c_i]) - 1 else 0
        else:
            df_data["result"].append(0)
    all_df_data.append(df_data)
        
print(f"All different period data len: {len(all_df_data)}", f"true: {all_df_data[2]['result'].count(1)}", f"false: {all_df_data[2]['result'].count(0)}")

C:\Users\Danila\AppData\Local\Temp\ipykernel_14436\1641628052.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


All different period data len: 5 true: 8672 false: 13097


# Add Strategy Signals To DF

In [7]:
from Strategies.StrategyAroon import StrategyAroon
from Strategies.StrategyStochRSI import StrategyStochRSI
from Strategies.StrategyEMA import StrategyEMA

# candle interval init (for this case is always 1 min)
candle_interval = CandleInterval.CANDLE_INTERVAL_1_MIN

# init all strategies
EMA_down = StrategyEMA()
EMA_up = StrategyEMA()
EMA_flat = StrategyEMA()
EMA_short_up = StrategyEMA()
EMA_short_down = StrategyEMA()

RSI_down = StrategyRSI()
RSI_up = StrategyRSI()
RSI_flat = StrategyRSI()
RSI_short_up = StrategyRSI()
RSI_short_down = StrategyRSI()

MACD_down = StrategyMACD()
MACD_up = StrategyMACD()
MACD_flat = StrategyMACD()
MACD_short_up = StrategyMACD()
MACD_short_down = StrategyMACD()

BB_down = StrategyBB()
BB_up = StrategyBB()
BB_flat = StrategyBB()
BB_short_up = StrategyBB()
BB_short_down = StrategyBB()

ST_down = StrategyST()
ST_up = StrategyST()
ST_flat = StrategyST()
ST_short_up = StrategyST()
ST_short_down = StrategyST()

StochRSI_down = StrategyStochRSI()
StochRSI_up = StrategyStochRSI()
StochRSI_flat = StrategyStochRSI()
StochRSI_short_up = StrategyStochRSI()
StochRSI_short_down = StrategyStochRSI()

Aroon_down = StrategyAroon()
Aroon_up = StrategyAroon()
Aroon_flat = StrategyAroon()
Aroon_short_up = StrategyAroon()
Aroon_short_down = StrategyAroon()

OBV_down = StrategyOBV()
OBV_up = StrategyOBV()
OBV_flat = StrategyOBV()
OBV_short_up = StrategyOBV()
OBV_short_down = StrategyOBV()

strategies_blocks = [[EMA_down, EMA_up, EMA_flat, EMA_short_up, EMA_short_down], 
                    [RSI_down, RSI_up, RSI_flat, RSI_short_up, RSI_short_down], 
                    [MACD_down, MACD_up, MACD_flat, MACD_short_up, MACD_short_down], 
                    [BB_down, BB_up, BB_flat, BB_short_up, BB_short_down],
                    [StochRSI_down, StochRSI_up, StochRSI_flat, StochRSI_short_up, StochRSI_short_down],
                    [ST_down, ST_up, ST_flat, ST_short_up, ST_short_down],
                    [Aroon_down, Aroon_up, Aroon_flat, Aroon_short_up, Aroon_short_down],
                    [OBV_down, OBV_up, OBV_flat, OBV_short_up, OBV_short_down]]

for strategy_block in strategies_blocks:
    strategy_block[0].initialize_moving_avg_container(candles_down[:strategy_block[0].history_candles_length])
    strategy_block[1].initialize_moving_avg_container(candles_up[:strategy_block[1].history_candles_length])
    strategy_block[2].initialize_moving_avg_container(candles_flat[:strategy_block[2].history_candles_length])
    strategy_block[3].initialize_moving_avg_container(candles_flat[:strategy_block[3].history_candles_length])
    strategy_block[4].initialize_moving_avg_container(candles_flat[:strategy_block[4].history_candles_length])

print(f"Strategy blocks len: {len(strategies_blocks)}")     

Strategy blocks len: 8


In [9]:
all_df_data_st = [{} for i in range(len(strategies_blocks[0]))]

for strategy_block in strategies_blocks:
    for str_i, strategy_type in enumerate(strategy_block):
        for candle_index in range(strategy_type.history_candles_length, len(all_candles[str_i])):
            params = strategy_type.get_candle_param(all_candles[str_i][candle_index])
            for param_name in params:
                if candle_index == strategy_type.history_candles_length:
                    all_df_data_st[str_i][param_name] = list()
                    [all_df_data_st[str_i][param_name].append(None) for i in range(strategy_type.history_candles_length)]
                all_df_data_st[str_i][param_name].append(params[param_name])
            
print(f"Small length check should be the length of candles: {len(all_df_data_st[2]['BB_lower'])}, candle len: {len(all_candles[2])}")

Small length check should be the length of candles: 21769, candle len: 21769


In [10]:
# convert to df
for i in range(len(all_df_data)):
    all_df_data[i] = pd.DataFrame(all_df_data[i])

for i in range(len(all_df_data)):
    for name in all_df_data_st[i]:
        all_df_data[i][name] = all_df_data_st[i][name]

biggest_gap = 0
for strategy_block in strategies_blocks:
    for strategy_type in strategy_block:
        if strategy_type.history_candles_length > biggest_gap:
            biggest_gap = strategy_type.history_candles_length

# delete first NaN elements
for i, df in enumerate(all_df_data):
    all_df_data[i] = df.drop(df.index[:biggest_gap])

concatenated_df = pd.DataFrame()
for i, data in enumerate(all_df_data):
    data.to_csv(f"./marked_data/result_with_parameters_{i}.csv", index=False)
    
all_df_data[0]

,index,result,close,open,high,low,volume,EMA_prev_diff,EMA_current_diff,prev_RSI,...,BB_lower,StochRSI_K,StochRSI_D,ST_upper,ST_lower,ART,aroon_up,aroon_low,OBV,OBV_MA
200,200,1,265.12,265.00,265.34,264.62,65474,-1.209035,-12.677693,53.919357,...,263.052341,2.558635,31.683117,109.181222,276.175918,1.905871,28.0,92.0,-26210098.0,-2.612715e+07
201,201,1,264.22,265.11,265.23,264.14,83902,-12.677693,-22.862078,93.194427,...,262.786101,0.000000,30.830239,109.181222,276.175918,1.918146,24.0,100.0,-26294000.0,-2.614953e+07
202,202,0,262.86,264.22,264.48,262.53,200461,-22.862078,-31.847178,92.658673,...,262.514976,0.000000,30.830239,109.181222,276.175918,1.986937,20.0,100.0,-26494461.0,-2.618962e+07
203,203,0,263.69,262.86,263.85,262.82,77133,-31.847178,-39.936275,91.852726,...,262.442282,17.887931,36.792883,109.181222,276.175918,1.989808,16.0,96.0,-26417328.0,-2.622111e+07
204,204,0,264.50,263.69,264.74,263.58,71947,-39.936275,-47.213396,91.896176,...,262.511790,35.344828,42.611848,109.181222,276.175918,2.001154,12.0,92.0,-26345381.0,-2.623554e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17373,17373,0,130.70,130.75,130.87,130.50,13611,-0.975887,-1.009690,53.147416,...,129.736794,41.176471,44.555729,109.181222,276.175918,1.557147,16.0,28.0,-51826678.0,-5.185185e+07
17374,17374,0,130.80,130.70,130.98,130.61,17154,-1.009690,-1.047542,53.147416,...,129.715555,44.444444,45.645054,109.181222,276.175918,1.544670,12.0,24.0,-51809524.0,-5.185044e+07
17375,17375,0,130.95,130.85,131.00,130.80,11660,-1.047542,-1.093248,53.347018,...,129.837038,48.717949,47.069555,109.181222,276.175918,1.521692,100.0,20.0,-51797864.0,-5.184793e+07
17376,17376,1,131.30,130.85,131.50,130.74,37603,-1.093248,-1.163004,53.646210,...,129.737552,56.521739,49.670819,109.181222,276.175918,1.537579,100.0,16.0,-51760261.0,-5.184203e+07
